Part 2
* Formatting data
* Profiling data continued
    * Missing data
    * Unique values and counts
    * De-duplicating
* Clean missing data
* Group By operations and reshaping data
* Combining data and merging (joining) data
* Rolling / window functions and lead/lag shift
* Custom functions - apply/applymap

In [ ]:
import os
import pandas as pd
import seaborn as sns

# custom module for this tutorial
import utils

In [ ]:
# downloading searborn datasets
utils.prep_example_data()

In [ ]:
# downloading trip history data from Divvy bikeshare
# docs: https://www.divvybikes.com/system-data

divvy_urls = ['https://divvy-tripdata.s3.amazonaws.com/Divvy_Trips_2019_Q1.zip',
       'https://divvy-tripdata.s3.amazonaws.com/Divvy_Trips_2019_Q2.zip',
       'https://divvy-tripdata.s3.amazonaws.com/Divvy_Trips_2019_Q3.zip',
       'https://divvy-tripdata.s3.amazonaws.com/Divvy_Trips_2019_Q4.zip']

# this may take 1-2 minutes. Downloading 4 CSV files.
for url in divvy_urls:
#     utils.process_url_zip(url=url, zip_name='divvy.zip', target_path='./data')
    print('')

In [ ]:
# check that data files are there

os.listdir('./data')

# Prep - load and profile data set

In [ ]:
divvy = pd.read_csv('./data/Divvy_Trips_2019_Q1.csv')

In [ ]:
# inspect top 5 rows
divvy.head()

In [ ]:
# check size and object types
divvy.info(memory_usage='deep')

In [ ]:
# get top 10 birthyears and check NAs
# pd.NaN is numpy object for null values
divvy['birthyear'].value_counts(dropna=False).head(10)

# Formatting data

In [ ]:
divvy['gender'].head()

### map()

In [ ]:
# map words to abbreviation

gender_map = {'Male': 'M', 'Female': 'F'}
divvy['gender2'] = divvy['gender'].map(gender_map)
# divvy['gender2'] = divvy['gender'].map({'Male': 'M', 'Female': 'F'})

divvy.head(3)

### Converting data types with `.astype(<type>)`

This throws an error...

`divvy['birthyear2'] = divvy['birthyear'].astype(int)`

```
ValueError                                Traceback (most recent call last)
<ipython-input-27-3b2a21753e03> in <module>
----> 1 divvy['birthyear2'] = divvy['birthyear'].astype(int)
...
ValueError: Cannot convert non-finite values (NA or inf) to integer
```

In [ ]:
# fill nulls with 0, then convert to int
    # we'll cover fillna() and dropna() later
    
divvy['birthyear2'] = divvy['birthyear'].fillna(0).astype(int)
divvy.head(3)

### String operations

### Extract streetnames

In [ ]:
# we will parse the station names
divvy['from_station_name'].head()

In [ ]:
# for pandas, remember to access .str attribute to make string methods available
    # else you will get this error --> `AttributeError: 'Series' object has no attribute 'split'`

divvy['from_station_name'].str.split(' & ').head()

In [ ]:
divvy['from_station_name'].str.split(' & ', expand=True).head()

In [ ]:
divvy[['from_x', 'from_y']] = divvy['from_station_name'].str.split(' & ', expand=True)
divvy[['from_x', 'from_y']].head()

In [ ]:
divvy.head()

### String search and regex pattern matching

In [ ]:
# search_bool is a Series of True/False's
# pass to bracket [] of dataframe for boolean indexing/filtering

# filter to where station contains Dearborn in name
search_bool = divvy['from_station_name'].str.contains('Dearborn')
divvy[search_bool].head()

In [ ]:
# use of not (~) operator
no_and_character_bool = ~(divvy['from_station_name'].str.contains('&'))
divvy[no_and_character_bool].loc[:, 'from_station_name'].head()

In [ ]:
# use regex search via Series.str.contains(<pattern>, regex=True)

regex_bool = divvy['from_station_name'].str.contains('Pkwy|Pl', regex=True)
divvy[regex_bool].head(3)

### Convert continous variables to categories (binning data)

If you want equal distribution of the items in your bins, use `qcut` . If you want to define your own numeric bin ranges, then use `cut`.

In [ ]:
# create age column
    # more on date formatting later

from datetime import datetime

current_year = datetime.now().year
print(f'current year is: {current_year}')

divvy['age'] = current_year - divvy['birthyear']
divvy[['birthyear', 'age']].head()

In [ ]:
divvy['age'].describe()

In [ ]:
# cut into 4 intervales of equal size

divvy['age_interval'] = pd.qcut(divvy['age'], q=4)
divvy['age_interval'].value_counts(dropna=False, normalize=True)

In [ ]:
divvy[['age', 'age_interval']].head()

In [ ]:
# cutting into quintiles
bin_labels = ['bottom third', 'middle third', 'upper third'] 
divvy['age_interval2'] = pd.qcut(divvy['age'], q=[0, .33, .66, 1], labels=bin_labels)

In [ ]:
# sample random 10
divvy[['age', 'age_interval2']].sample(10)

In [ ]:
# intervals of eqaul size intervals -- but un-equal distributions

pd.cut(divvy['age'], bins=4).value_counts(dropna=False, normalize=True)

### Converting datetimes

Notes
* pandas/numpy datetime64 data type is more efficient than native Python datetime vie `datetime` standard library
* try to convert to pandas datetime64 if you're doing computations on many dates (ex: dates in a dataset)
* docs: https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html

In [ ]:
divvy['start_time'] = pd.to_datetime(divvy['start_time'])

In [ ]:
divvy.dtypes

In [ ]:
# now, using `.dt.` attribute, we can access datetime-like attributes too

divvy['start_day'] = divvy['start_time'].dt.day
divvy['start_month'] = divvy['start_time'].dt.month
divvy['start_hour'] = divvy['start_time'].dt.hour

divvy[['start_time', 'start_day', 'start_month', 'start_hour']].head()

In [ ]:
# now we can plot frequency by hour

divvy['start_hour'].value_counts(normalize=True)\
                   .sort_index()\
                   .plot(kind='bar')

# Calculated fields

In [ ]:
# tripduration is a string... to do math, we want it to be a float or int

divvy['tripduration'].head()

In [ ]:
# convert string to float: replace commas, replace '.0', then cast to float
divvy['tripduration'] = divvy['tripduration'].str.replace(',', '')\
                                             .str.replace('.0','')\
                                             .replace('','0')\
                                             .astype(int)

# make new tripduration field that converts minutes to hours
divvy['tripduration_hrs'] = divvy['tripduration'] / 60

In [ ]:
divvy[['tripduration', 'tripduration_hrs']].head()

# Profiling data

In [ ]:
# .describe() can be used on sub-selection
# .T for transpose

divvy[['gender', 'age']].describe(include='all').T

In [ ]:
divvy.describe(include='all')

In [ ]:
# unique values via drop_duplicates or numpy .unique()

divvy['from_station_name'].drop_duplicates()
# divvy['from_station_name'].unique()

In [ ]:
# number of unique values

divvy['from_station_name'].nunique()
# divvy['from_station_name'].drop_duplicates().shape[0]

### De-duplication

In [ ]:
names = pd.DataFrame([['John', 'Doe', '123 Main St'],
                     ['John', 'Doe', '999 Wall St'],
                     ['John', 'Doe', '123 Main St'],
                     ['Jane', 'Dee', '1 Pennsylvania'],
                     ['Jane', 'Dee', 'Dearborn / Erie'],
                     ['Mike', 'Jones', 'Palmer House']], columns=['first', 'last', 'address'])

names

In [ ]:
# dedupe at level of first-last name
# arbitrarily keep first address (can sort if applicable)

deduped_names = names.drop_duplicates(subset=['first', 'last'], keep='first')
deduped_names

### Identifying duplicates

In [ ]:
names

In [ ]:
# dupes across all columns
# John Doe @ 123 Main St exists twice in original
    # Jane Dee exists twice but at different addresses

names[names.duplicated()]

In [ ]:
# dupes across subset of columns

dupes = names[names.duplicated(subset=['first', 'last'])]
dupes

In [ ]:
# unique list of first-last names that are duplicated

names.iloc[dupes.index].loc[:,['first', 'last']].drop_duplicates()

# Missing data


# Group By operations and reshaping data
* df.groupby()
* df.pivot_table()
* df.melt()
* pd.crosstab()
* df.T

* note: caution on NAs. Fill NAs in grouping columns prior to grouping.

# Combining data and merging (joining) data
* pd.concat()
* df.merge()

# Rolling / window functions and lead/lag shift


# Other

```python
# # formatted

# import matplotlib.pyplot as plt
# sns.set()

# fig, ax = plt.subplots(figsize=(15,5))
# divvy['start_hour'].value_counts(normalize=True)\
#                    .sort_index()\
#                    .plot(kind='bar')

# ax.set_title('Bike Ridership by Hour of Day', fontsize='large')
# ax.set_xlabel('Hour of Day')
# ax.set_ylabel('Percentage of Riders')
# xlabels = divvy['start_hour'].unique().tolist()
# plt.xticks(rotation='horizontal')
```